In [5]:
!pytest basic/failure_demo.py

============================= test session starts ==============================
platform linux -- Python 3.13.1, pytest-8.4.1, pluggy-1.6.0
rootdir: /home/florian/talks/trainings/2025-07-ep2025/demo/code
configfile: pytest.ini
plugins: anyio-4.9.0, md-0.2.0, xdist-3.8.0, pytest_httpserver-1.1.3, mock-3.14.1, reportlog-0.4.0, instafail-0.5.0, hypothesis-6.135.20, rich-0.2.0, cov-6.2.1
collected 8 items                                                              

basic/failure_demo.py FFFFFFFF                                           [100%]

=================================== FAILURES ===================================
_________________________________ test_eq_text _________________________________

    def test_eq_text():
>       assert "spam" == "eggs"
E       AssertionError: assert 'spam' == 'eggs'
E         
E         - eggs
E         + spam

basic/failure_demo.py:2: AssertionError
_____________________________ test_eq_similar_text _____________________________

    def test_eq

In [6]:
!pytest basic/test_traceback.py

============================= test session starts ==============================
platform linux -- Python 3.13.1, pytest-8.4.1, pluggy-1.6.0
rootdir: /home/florian/talks/trainings/2025-07-ep2025/demo/code
configfile: pytest.ini
plugins: anyio-4.9.0, md-0.2.0, xdist-3.8.0, pytest_httpserver-1.1.3, mock-3.14.1, reportlog-0.4.0, instafail-0.5.0, hypothesis-6.135.20, rich-0.2.0, cov-6.2.1
collected 2 items                                                              

basic/test_traceback.py F.                                               [100%]

=================================== FAILURES ===================================
_________________________________ test_divide __________________________________

    def test_divide():
        # This will raise ZeroDivisionError
>       assert calc(2, 0, "/") == 0
               ^^^^^^^^^^^^^^^

basic/test_traceback.py:5: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

a = 2, b = 0, op = '/'

    def calc(a, b,

# raises

In [7]:
def parse_pos_int(s: str) -> int:
    n = int(s)
    if n < 0:
        raise ValueError(f"No negativity allowed, but got {n}")
    return n

In [8]:
parse_pos_int("5")

5

In [9]:
parse_pos_int("-2")

ValueError: No negativity allowed, but got -2

In [10]:
parse_pos_int("a")

ValueError: invalid literal for int() with base 10: 'a'

In [17]:
%%ipytest
import pytest

def test_good():
    assert parse_pos_int("5") == 5

def test_negative():
    with pytest.raises(ValueError, match=r"No negativity allowed"):
        parse_pos_int("-2")

def test_invalid():
    with pytest.raises(ValueError, match=r"invalid literal for int"):
        parse_pos_int("a")

======================================= test session starts ========================================
collected 3 items

t_47e85ff7049145cbbfc6f9b3aeff836e.py ...                                                    [100%]

======================================== 3 passed in 0.00s =========================================


In [18]:
%%ipytest
import pytest

from rpncalc.rpn_v2 import RPNCalculator
from rpncalc.utils import Config


# exercise: [fixtures]
@pytest.fixture
def rpn() -> RPNCalculator:
    return RPNCalculator(Config())


@pytest.mark.parametrize(
    "op, expected",
    [
        ("+", 3),
        ("-", -1),
        ("*", 2),
        ("/", 0.5),
    ],
)
def test_operations(op: str, expected: float, rpn: RPNCalculator):
    rpn.stack = [1, 2]
    rpn.evaluate(op)
    assert rpn.stack == [expected]

# monkeypatch

In [28]:
%%ipytest -v

import pytest

from rpncalc.rpn_v2 import RPNCalculator
from rpncalc.utils import Config


@pytest.fixture
def rpn() -> RPNCalculator:
    return RPNCalculator(Config())


@pytest.mark.parametrize("inputs, stack, expected_out, expected_err", [
    pytest.param(["1", "2", "+", "q"], [3], "3.0\n", "", id="add"),
    pytest.param(["1", "2", "p", "q"], [1, 2], "[1.0, 2.0]\n", "", id="print"),
    pytest.param(["1", "0", "/", "q"], [], "", "Division by zero\n", id="div0"),
])
def test_run(
    rpn: RPNCalculator,
    monkeypatch: pytest.MonkeyPatch,
    capfd: pytest.CaptureFixture[str],
    inputs: list[str],
    stack: list[float],
    expected_out: str,
    expected_err: str,
):
    monkeypatch.setattr(rpn, "get_inputs", lambda: inputs)
    rpn.run()
    assert rpn.stack == stack
    out, err = capfd.readouterr()
    assert out == expected_out
    assert err == expected_err

======================================= test session starts ========================================
collecting ... collected 3 items

t_47e85ff7049145cbbfc6f9b3aeff836e.py::test_run[add] PASSED                                  [ 33%]
t_47e85ff7049145cbbfc6f9b3aeff836e.py::test_run[print] PASSED                                [ 66%]
t_47e85ff7049145cbbfc6f9b3aeff836e.py::test_run[div0] PASSED                                 [100%]

======================================== 3 passed in 0.01s =========================================


# tmp_path

In [30]:
%%ipytest
from pathlib import Path

import pytest

from rpncalc.utils import Config


@pytest.fixture
def config():
    return Config()


@pytest.fixture
def ini_path(tmp_path: Path) -> Path:
    return tmp_path / "rpncalc.ini"


@pytest.fixture
def example_ini(ini_path: Path) -> Path:
    # creates rpncalc.ini with pathlib
    ini_path.write_text(
        "[rpncalc]\n"
        "prompt = rpn>\n")
    return ini_path


# exercise: [load-save]

def test_config_load(
    example_ini: Path, config: Config
):
    assert config.prompt == ">"  # sanity check
    config.load(example_ini)
    assert config.prompt == "rpn>"


def test_config_save(
    ini_path: Path, config: Config
):
    assert not ini_path.exists()  # sanity check
    config.save(ini_path)
    assert ini_path.exists()


======================================= test session starts ========================================
collected 2 items

t_47e85ff7049145cbbfc6f9b3aeff836e.py ..                                                     [100%]

======================================== 2 passed in 0.00s =========================================


In [31]:
%pycat fixtures/test_fixture_scope_reset.py

import pytest
import time
from rpncalc.rpn_v2 import RPNCalculator, Config


@pytest.fixture(scope="module")
def rpn_instance() -> RPNCalculator:
    time.sleep(2)
    return RPNCalculator(Config())


@pytest.fixture
def rpn(
    rpn_instance: RPNCalculator,
) -> RPNCalculator:
    rpn_instance.stack.clear()
    return rpn_instance


def test_a(rpn: RPNCalculator):
    rpn.stack.append(42)
    assert rpn.stack == [42]


def test_b(rpn: RPNCalculator):
    assert not rpn.stack
